<a href="https://colab.research.google.com/github/khyatidoshi/HumorGeneration/blob/main/Humor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import torch
import torchtext as tt
from torch.utils.data import DataLoader
from torch import nn, optim
from collections import Counter
import torch.nn.functional as F

In [3]:
joke_dataset = pd.read_csv('new_joke_data.csv')
joke_dataset.head()

,ID,Joke
0,0,"[me narrating a documentary about narrators] ""..."
1,1,Telling my daughter garlic is good for you. Go...
2,2,I've been going through a really rough period ...
3,3,"If I could have dinner with anyone, dead or al..."
4,4,Two guys walk into a bar. The third guy ducks.


In [4]:
sequence_length=128

In [5]:
class Dataset():
  def __init__(self, dataset, sequence_length=128):
    self.sequence_length = sequence_length
    self.words = self.load_words(dataset)
    self.uniq_words = self.get_uniq_words()
    self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
    self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}
    self.words_indexes = [self.word_to_index[w] for w in self.words]

  def load_words(self, dataset):
    text = " ".join(dataset['Joke'])
    return text.split(' ')

  def get_uniq_words(self):
    word_counts = Counter(self.words)
    return sorted(word_counts, key=word_counts.get, reverse=True)

  def __len__(self):
    return len(self.words_indexes) - self.sequence_length

  def __getitem__(self, index):
    return (
      torch.tensor(self.words_indexes[index:index + self.sequence_length]),
      torch.tensor(self.words_indexes[index + 1:index + self.sequence_length + 1]),
    )

df = Dataset(joke_dataset,sequence_length)

In [6]:
class Model(nn.Module):
    def __init__(self, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(Model, self).__init__()
        self.ntoken = ntoken
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        # self.rnn = nn.Sequential(
        #     nn.LSTM(ninp, nhid, nlayers, dropout=dropout),
        #     nn.LSTM(nhid, nhid, nlayers, dropout=dropout),
        # )
        self.rnn = getattr(nn, 'LSTM')(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)
        self.init_weights()
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.encoder.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.bias)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, input, hidden):
        print(" In forward")
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = F.leaky_relu(self.decoder(output))
        return decoded, hidden

    def init_state(self, bsz):
        weight = next(self.parameters())
        return (weight.new_zeros(self.nlayers , bsz, self.nhid),
                weight.new_zeros(self.nlayers , bsz, self.nhid))

In [7]:
# df = Dataset(joke_dataset,sequence_length)
# ntoken = len(df.uniq_words)

model = Model(len(df.uniq_words), sequence_length , nhid=1024, nlayers=3, dropout=0.5)



In [8]:
# Usage
# batch_size = 128
num_epochs = 5
learning_rate = 0.001

# train_model(model, df, batch_size, num_epochs, learning_rate)


In [ ]:
# Initialize the model
# model = Model(ntoken, sequence_length, nhid=512, nlayers=4, dropout=0.5)

# Set the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

train_loader = DataLoader(df, batch_size=sequence_length, shuffle=True)

# Train the model in a loop
for epoch in range(num_epochs):
    # for inputs, targets in train_loader:
    for i, data in enumerate(train_loader):
        input, target = data[0],data[1]
        # print("data : ",data)
        # Forward pass
        hidden = model.init_state(sequence_length)
        # print(" INPUT : ",input.size())
        # print(" INPUT REST : ",hidden)
        output, hidden = model(input,hidden)
        print(" HERE output: ",output)
        print(" HERE target: ",target)
        # Calculate the loss
        loss = loss_fn(output, target)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print the loss every 100 steps
        if i % 100 == 0:
            print('Epoch {} | Step {} | Loss {}'.format(epoch, i, loss.item()))

# Save the trained model
torch.save(model, 'model.pt')

 In forward
